In [1]:
# 코드 실행 후 마지막 ngrok url을 4_multi-llm-gradio.ipynb에 적용

In [1]:
#!pip install pyngrok==4.1.1.
!curl -fsSL https://ollama.com/install.sh | sh
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers
!pip install pyngrok


>>> Downloading ollama...
######################################################################## 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
Failed to connect to bus: No such file or directory
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Hit:1 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease
Hit:2 https://download.docker.com/linux/ubuntu focal InRelease                 
Hit:3 https://cli.github.com/packages stable InRelease                         
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease               
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease                         
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease     

In [2]:
from pyngrok import ngrok
with open('llm_ngrok_cred.txt', 'r') as f:
  token = [r.strip() for r in f.readlines()]
#print(token)
ngrok.set_auth_token(token[0])

In [3]:
import os
import asyncio

# set LD_LIBRARY_PATH
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

In [4]:
# 이 부분 코드 실행하고 나서, 처음에 등장하는 ngrok url 주소를 따로 저장해주셔야 합니다.
# 예시 : 아래에서 https://71e8-35-237-75-201.ngrok-free.app
#  t=2024-05-24T03:29:01+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://localhost:11434 url=https://71e8-35-237-75-201.ngrok-free.app


async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )
from IPython.display import clear_output
clear_output()


await asyncio.gather(
run_process(['ollama', 'serve']),
run_process(['ngrok', 'http', '--log', 'stderr', '11434', '--host-header="localhost:11434"']),
run_process(['ollama', 'pull', 'gemma:7b-instruct']), # 사용할 모델 pull하는 부분
run_process(['ollama', 'pull', 'llama3:8b']), # 사용할 모델 pull하는 부분

)


>>> starting ollama serve
>>> starting ngrok http --log stderr 11434 --host-header="localhost:11434"
>>> starting ollama pull gemma:7b-instruct
>>> starting ollama pull llama3:8b
2024/06/15 04:20:32 routes.go:1011: INFO server config env="map[OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_KEEP_ALIVE: OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:1 OLLAMA_MAX_QUEUE:512 OLLAMA_MAX_VRAM:0 OLLAMA_MODELS:/teamspace/studios/this_studio/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_TMPDIR:]"
time=2024-06-15T04:20:32.011Z level=INFO source=images.go:725 msg="total blobs: 12"
time=2024-06-15T04:20:32.012Z level=INFO source=images.go:732 ms

t=2024-06-15T04:20:32+0000 lvl=info msg="client session established" obj=tunnels.session
t=2024-06-15T04:20:32+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2024-06-15T04:20:32+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://localhost:11434 url=https://89f6-44-211-63-248.ngrok-free.app
time=2024-06-15T04:20:35.026Z level=INFO source=payload.go:44 msg="Dynamic LLM libraries [rocm_v60002 cpu cpu_avx cpu_avx2 cuda_v11]"
time=2024-06-15T04:20:35.606Z level=INFO source=types.go:71 msg="inference compute" id=GPU-d17865ec-cfb9-c4a4-9d72-3e2e69b0b46f library=cuda compute=7.5 driver=12.2 name="Tesla T4" total="14.7 GiB" available="14.6 GiB"
time=2024-06-15T04:20:35.606Z level=INFO source=types.go:71 msg="inference compute" id=GPU-5dc7b06a-35c5-f7a8-416a-7ca5d3d0c84d library=cuda compute=7.5 driver=12.2 name="Tesla T4" total="14.7 GiB" available="14.6 GiB"
time=2024-06-15T04:20:35.606Z level=INFO source=types.go:71 msg="inference compute" id=GPU-24